# Desafio 6

## Importando bibliotecas e pacotes

In [ ]:
from keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
import IPython.display as display
from keras.datasets import mnist
import tensorflow_hub as hub
from keras import layers
import matplotlib as mpl
import tensorflow as tf
import seaborn as sns
from PIL import Image
import numpy as np
import functools
import keras
import time
import PIL
import os

## Parte 1 - Autoencoder

Utilize o código a seguir para remover ruídos de imagens do dataset MNIST

Link: [google_colab](https://colab.research.google.com/drive/1XeZT0wwxepsip0HMasx4YyGiav_aXgU7?usp=sharing)

### Carregando o dataset e dividindo os dados entre treino e teste

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### Normalizando os dados

In [ ]:
x_train = x_train.astype("float32") / 255.
x_test = x_test.astype("float32") / 255.

### Transpondo as matrizes

In [ ]:
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

### Parâmetros para gerar imagens com ruídos

In [ ]:
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(
    loc = 0.0,
    scale = 1.0,
    size = x_train.shape
)
x_test_noisy = x_test + noise_factor * np.random.normal(
    loc = 0.0,
    scale = 1.0,
    size = x_test.shape
)

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

### Gerandos as imagens com ruídos e imprimindo o resultado

In [ ]:
max_iter = 10
plt.figure(figsize = (20, 2))

for i in range(1, max_iter + 1):
    axis = plt.subplot(1, max_iter, i)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    axis.get_xaxis().set_visible(False)
    axis.get_yaxis().set_visible(False)

plt.show()

### Gerando modelo para realizar a limpeza dos ruídos

In [ ]:
# formatando imagem para entrada
input_img = keras.Input(shape = (28, 28, 1))

# camada de entrada e
# uma camada de convolucao 2d
x = layers.Conv2D(
    32,
    (3, 3),
    activation = "relu",
    padding = "same"
)(input_img)

# camada que realiza um maxpooling 2d
x = layers.MaxPooling2D(
    (2, 2),
    padding = "same"
)(x)

# segunda camada de convolucao 2d
x = layers.Conv2D(
    32,
    (3, 3),
    activation = "relu",
    padding = "same"
)(x)

# segunda camada de maxpooling 2d
encoded = layers.MaxPooling2D(
    (2, 2),
    padding = "same"
)(x)


# terceira camada de convolucao 2d
x = layers.Conv2D(
    32,
    (3, 3),
    activation = "relu",
    padding = "same"
)(encoded)

# uma cadama de upsampling
x = layers.UpSampling2D((2, 2))(x)

# quarta camada de convolucao 2d
x = layers.Conv2D(
    32,
    (3, 3),
    activation = "relu",
    padding = "same"
)(x)

# segunda camada de upsampling
x = layers.UpSampling2D((2, 2))(x)

# camada de decodificacao e
# camada de saida
decoded = layers.Conv2D(
    1,
    (3, 3),
    activation = "sigmoid",
    padding = "same"
)(x)

# empacotando todo o modelo
autoencoder = keras.Model(input_img, decoded)

# compilando o modelo
autoencoder.compile(optimizer = "adam", loss = "binary_crossentropy")

### Realizando o fit()

In [ ]:
autoencoder.fit(
    x_train_noisy,
    x_train,
    epochs = 50,
    batch_size = 256,
    shuffle = True,
    validation_data = (x_test_noisy, x_test),
    callbacks = [
        TensorBoard(
            histogram_freq = 0,
            write_graph = False
        )
    ]
)

## Parte 2 - GAN

### Estilo neural

Essa GAN aplica estilo de um artista em uma imagem, altere as imagens e estilos, assim como outros parâmetros que queira e gere uma arte digital.

Link: [google_colab](https://colab.research.google.com/drive/1l2QRo4KOoQE3Bc5HvYem7g64h3wasQnx?usp=sharing)

### Configurando ambiente

In [ ]:
os.environ["TFHUB_MODEL_LOAD_FORMAT"] = "COMPRESSED"
mpl.rcParams["figure.figsize"] = (12, 12)
mpl.rcParams["axes.grid"] = False

### Função *tensor_to_image*

In [11]:
def tensor_to_image(tensor):
    tensor = tensor * 255
    tensor = np.array(tensor, dtype = np.uint8)
  
    if np.ndim(tensor) > 3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    
    return PIL.Image.fromarray(tensor)


### Função *load_img*

In [ ]:
def load_img(path_to_img):
    max_dim = 512
    
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    
    return img


### Função *show_image*

In [ ]:
def show_image(image, title = None):
    if len(image.shape) > 3:
        image = tf.squeeze(image, axis = 0)

    plt.imshow(image)
    
    if title:
        plt.title(title)


### importando algumas imagens

In [ ]:
content_path = tf.keras.utils.get_file(
    "YellowLabradorLooking_new.jpg",
    "https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg"
)
style_path = tf.keras.utils.get_file(
    "kandinsky5.jpg",
    "https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg"
)

### Imprimindo as imagens

In [ ]:
content_image = load_img(content_path)
style_image = load_img(style_path)

plt.subplot(1, 2, 1)
show_image(content_image, "Content Image")

plt.subplot(1, 2, 2)
show_image(style_image, "Style Image")

### Imagem estilizada

In [ ]:
hub_model = hub.load("https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2")
stylized_image = hub_model(
    tf.constant(content_image),
    tf.constant(style_image)
)[0]
tensor_to_image(stylized_image)